<a href="https://colab.research.google.com/github/ililililililililililililililil/INI/blob/ini_pytorch/ini_7_1_Tips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Maximum Likehood Estimation (MLE)

#100번 실험했을 때 (n=100)
#class1이 나올 확률이 27이면 (k=27)
#확률분포를 결정하는 파라미터 값(세타)을 구할 수 있다. (구하는 방법은 다양)
#위 세타에 대한 함수를 구할 수 있다 -> Likehood

#Likehood y값이 최대가 되는 지점을 찾는다.
#그 x값이 방금 관찰한 observation을 가장 잘 설명하는 세타이다.

#Gradient Descent 알고리즘으로도 치대값을 구할 수 있다.
#(이전에 배운 최소 찾는 내용과 같다)

#Maximum Likehood Estimation은 숙명적으로 overfitting이 따른다.
#overfitting 최소화: 훈련 데이터와 테스트 데이터를 나눠서 훈련 후에 얼마나 잘 학습되었는지 검증
#테스트 데이터에 대해서도 overfitting되는 것을 막기 위해 Dev Set도 사용한다.
#(트레이닝 셋으로 훈련하고 Dev(Validation Set)로 검증하고 테스트 셋에서 테스트한다)

#그 외 overfitting 막기
#- 데이터 많이 모으기
#- features(특징) 적게 사용
#- Regularization (Early Stoppint: Validation Loss가 더 이상 낮아지지 않을 때)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# For reproducibility
torch.manual_seed(1)

In [4]:
# Training and Test Dataset

x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                             ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [5]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

In [8]:
#Model

class SoftmaxClassifierModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 3)
  def forward(self, x):
    return self.linear(x)

In [9]:
model = SoftmaxClassifierModel()

In [10]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [15]:
# Training

def train(model, optimizer, x_train, y_train):
  nb_epochs = 20
  for epoch in range(nb_epochs):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

In [16]:
# Test (Validation)

def test(model, optimizer, x_test, y_test):
  prediction = model(x_test)
  predicted_classes = prediction.max(1)[1]
  correct_count = (predicted_classes == y_test).sum().item()
  cost = F.cross_entropy(prediction, y_test)

  print('Accuracy: {}% Cost: {:.6f}'.format(
        correct_count / len(y_test) * 100, cost.item()
    ))

In [17]:
# Run

train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.199645
Epoch    1/20 Cost: 1.142985
Epoch    2/20 Cost: 1.117769
Epoch    3/20 Cost: 1.100901
Epoch    4/20 Cost: 1.089523
Epoch    5/20 Cost: 1.079872
Epoch    6/20 Cost: 1.071320
Epoch    7/20 Cost: 1.063325
Epoch    8/20 Cost: 1.055720
Epoch    9/20 Cost: 1.048377
Epoch   10/20 Cost: 1.041245
Epoch   11/20 Cost: 1.034285
Epoch   12/20 Cost: 1.027478
Epoch   13/20 Cost: 1.020813
Epoch   14/20 Cost: 1.014279
Epoch   15/20 Cost: 1.007872
Epoch   16/20 Cost: 1.001586
Epoch   17/20 Cost: 0.995419
Epoch   18/20 Cost: 0.989365
Epoch   19/20 Cost: 0.983424


In [18]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.386150


In [19]:
# Learning Rate
# Learning Rate이 너무 크면 diverge 하면서 cost가 점점 늘어난다 (overshooting)

model = SoftmaxClassifierModel()

In [21]:
optimizer = optim.SGD(model.parameters(), lr=1e5)

In [22]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.750000
Epoch    2/20 Cost: 1279135.250000
Epoch    3/20 Cost: 1198378.875000
Epoch    4/20 Cost: 1098825.750000
Epoch    5/20 Cost: 1968197.750000
Epoch    6/20 Cost: 284763.250000
Epoch    7/20 Cost: 1532260.250000
Epoch    8/20 Cost: 1651503.750000
Epoch    9/20 Cost: 521878.593750
Epoch   10/20 Cost: 1397263.250000
Epoch   11/20 Cost: 750986.375000
Epoch   12/20 Cost: 918691.375000
Epoch   13/20 Cost: 1487888.250000
Epoch   14/20 Cost: 1582260.250000
Epoch   15/20 Cost: 685818.125000
Epoch   16/20 Cost: 1140048.875000
Epoch   17/20 Cost: 940566.375000
Epoch   18/20 Cost: 931638.250000
Epoch   19/20 Cost: 1971322.750000


In [23]:
# Learning Rate
# Learning Rate이 너무 작으면 cost가 거의 줄어들지 않는다

model = SoftmaxClassifierModel()

In [24]:
optimizer = optim.SGD(model.parameters(), lr=1e-10)

In [25]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


In [26]:
# Learning Rate
#적절한 숫자로 시작해 발산하면 작게, cost가 줄어들지 않으면 크게 조정하자

model = SoftmaxClassifierModel()

In [27]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [28]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.341574
Epoch    1/20 Cost: 1.198802
Epoch    2/20 Cost: 1.150877
Epoch    3/20 Cost: 1.131977
Epoch    4/20 Cost: 1.116242
Epoch    5/20 Cost: 1.102514
Epoch    6/20 Cost: 1.089676
Epoch    7/20 Cost: 1.077479
Epoch    8/20 Cost: 1.065775
Epoch    9/20 Cost: 1.054511
Epoch   10/20 Cost: 1.043655
Epoch   11/20 Cost: 1.033187
Epoch   12/20 Cost: 1.023091
Epoch   13/20 Cost: 1.013356
Epoch   14/20 Cost: 1.003968
Epoch   15/20 Cost: 0.994917
Epoch   16/20 Cost: 0.986189
Epoch   17/20 Cost: 0.977775
Epoch   18/20 Cost: 0.969661
Epoch   19/20 Cost: 0.961836


In [29]:
# Data Preprocessing (데이터 전처리)

x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]
                             ])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [30]:
mu = x_train.mean(dim=0)

In [31]:
sigma = x_train.std(dim=0)

In [32]:
norm_x_train = (x_train - mu) / sigma

In [33]:
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


In [34]:
# Training with Preprocessed Data

class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)
  
  def forward(self, x):
    return self.linear(x)

In [35]:
model = MultivariateLinearRegressionModel()

In [36]:
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [37]:
def train(model, optimizer, x_train, y_train):
  nb_epochs = 20
  for epoch in range(nb_epochs):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.mse_loss(prediction, y_train)

    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

In [38]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29785.089844
Epoch    1/20 Cost: 18906.166016
Epoch    2/20 Cost: 12054.673828
Epoch    3/20 Cost: 7702.029785
Epoch    4/20 Cost: 4925.733398
Epoch    5/20 Cost: 3151.632812
Epoch    6/20 Cost: 2016.996094
Epoch    7/20 Cost: 1291.051270
Epoch    8/20 Cost: 826.505249
Epoch    9/20 Cost: 529.207397
Epoch   10/20 Cost: 338.934174
Epoch   11/20 Cost: 217.153564
Epoch   12/20 Cost: 139.206757
Epoch   13/20 Cost: 89.313782
Epoch   14/20 Cost: 57.375465
Epoch   15/20 Cost: 36.928429
Epoch   16/20 Cost: 23.835773
Epoch   17/20 Cost: 15.450401
Epoch   18/20 Cost: 10.077809
Epoch   19/20 Cost: 6.633700
